In [30]:
import  numpy as np
import pandas as pd
data=np.load('data.npz')

In [31]:
# 节点特征
node_features=data['x']
# 节点类别
node_class=data['y']
# 边特征
edge_index=data['edge_index']
edge_type=data['edge_type']

In [32]:
node_class=data['y'][(data['y']==0)| (data['y']==1)]

In [33]:
train_mask=data['train_mask']
test_mask=data['test_mask']

In [34]:
count_0 = (node_class == 0).sum().item()
count_1 = (node_class == 1).sum().item()
count_2= (node_class == 2).sum().item()
count_3= (node_class == 3).sum().item()

In [35]:
import torch
import os
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

In [36]:

print(torch.__version__)

2.1.1+cu118


In [37]:
torch.cuda.device_count()

1

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
os.environ['CUDA_VISIBLE_DEVICE']='0'
# 转换为pytorch张量



Device: cuda


In [39]:
x = torch.tensor(node_features, dtype=torch.float).contiguous()
y = torch.tensor(node_class, dtype=torch.int64)
edge_index = torch.tensor(edge_index.transpose(), dtype=torch.int64).contiguous()
edge_type = torch.tensor(edge_type, dtype=torch.float)
train_mask = torch.tensor(train_mask, dtype=torch.int64)
test_mask = torch.tensor(test_mask, dtype=torch.int64)

In [40]:
data=Data(x=x,edge_index=edge_index,edge_attr=edge_type,y=y)
data.train_mask=train_mask
data.test_mask=test_mask

In [29]:
from torch_geometric.nn import GCNConv
from sklearn.metrics import classification_report
import torch.nn.functional as F

# 定义 GCN 模型
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)  # 第一层，特征到16维
        self.conv2 = GCNConv(16, num_classes)   # 第二层，16维到类别数

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)  # ReLU激活函数
        x = F.dropout(x, training=self.training)  # Dropout

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)  # 使用 log_softmax 作为输出


# 初始化模型
model = GCN(num_features=17, num_classes=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)
# 训练模型
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# 训练循环
for epoch in range(200):
    loss = train()
    print(f'Epoch {epoch}, Loss: {loss:.4f}')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
